In [7]:
import pandas as pd
import numpy as np
import json
import os

In [8]:
folder_path = r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQD\Destructive SWAP\RY-2000-30"
cutoff = 16
folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [9]:
all_data = []
potentials = []
e0 = []
e1 = []
e2 = []
ratios = []

for f in folders:
    
    data_path = folder_path + "\\{}\\{}_{}.json".format(f,f, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    success_i = [i for i, success in enumerate(data['success']) if all(success)]
    successful_results = [data['results'][i] for i in success_i]
    energies =np.sort(successful_results)

    for e in energies:
        
        potentials.append(f)
        e0.append(e[0])
        e1.append(e[1])
        e2.append(e[2])

        r = abs((e[2] - e[1]) / (e[2] - e[0]))
        ratios.append(r)

df = {
"potential": potentials,
    "cutoff": cutoff,
    "e0": e0,
    "e1": e1,
    "e2": e2,
    "ratios": ratios
    }
    


In [10]:
df = pd.DataFrame(df)
df

,potential,cutoff,e0,e1,e2,ratios
0,AHO,16,-0.001167,1.659348,1.690264,0.018278
1,AHO,16,-0.001167,1.637938,1.691595,0.031698
2,AHO,16,-0.001167,1.662896,1.671748,0.005292
3,AHO,16,-0.001167,1.657687,1.677726,0.011936
4,AHO,16,-0.001167,1.664771,1.685837,0.012488
...,...,...,...,...,...,...
288,QHO,16,0.000000,0.982096,0.998958,0.016880
289,QHO,16,0.000000,0.957878,0.982292,0.024854
290,QHO,16,0.000000,0.958724,0.992318,0.033854
291,QHO,16,0.000000,0.983073,0.986328,0.003300


In [11]:
median = []
mean = []
min = []
potentials = []

for f in folders:

    potentials.append(f)
    data = df[df['potential'] == f]
    median.append(np.median(data['ratios']))
    mean.append(np.mean(data['ratios']))
    min.append(np.min(data['ratios']))

aggdf = {
    "potential": potentials,
    "cutoff": cutoff,
    "median": median,
    "mean": mean,
    "min": min,
}

# RY-CNOT

In [6]:
#2000 iters, 30 swaps
pd.DataFrame(aggdf)

,potential,cutoff,median,mean,min
0,AHO,16,0.011352,0.050834,0.000012
1,DW,16,0.990926,0.990934,0.975749
2,QHO,16,0.015830,0.017396,0.000065


# RY

In [6]:
#500 iters, 20 swaps
pd.DataFrame(aggdf)

,potential,cutoff,median,mean,min
0,AHO,16,0.021166,0.023772,0.000614
1,DW,16,0.947386,0.946651,0.914324
2,QHO,16,0.022510,0.022911,0.000200


In [12]:
#2000 iters, 30 swaps
pd.DataFrame(aggdf)

,potential,cutoff,median,mean,min
0,AHO,16,0.016820,0.022630,0.001048
1,DW,16,0.952525,0.951865,0.925315
2,QHO,16,0.013019,0.015993,0.000199
